### generate random token for `ego_pose`

In [2]:
import os
import json
import numpy as np
from dataset.token_gen import generate_random_token

class Ego_original:
    def __init__(self, timestamp, translation, rotation):
        self.timestamp = timestamp
        self.translation = translation
        self.rotation = rotation

class Sample_data:
    def __init__(self, ego_pose_token, timestamp):
        self.ego_pose_token = ego_pose_token
        self.timestamp = timestamp

def read_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def find_closest_timestamp(target_timestamp, ego_data):
    timestamps = np.array([ego['timestamp'] for ego in ego_data], dtype=np.float64)
    closest_index = np.argmin(np.abs(timestamps - target_timestamp))
    return ego_data[closest_index]

if __name__ == "__main__":
    # 文件路径
    ego_n005_path = './dataset/MB_dataset2/egopose_n005.json'
    sample_data_path = './dataset/MB_dataset2/mb_test/sample_data.json'

    # 读取 JSON 文件
    ego_data = read_json(ego_n005_path)['ego_poses']
    sample_data = read_json(sample_data_path)

    # 提取 sample_data 中的所有 timestamp，并去重和转换为字符串，然后排序
    unique_timestamps = sorted({str(np.float32(sample['timestamp'])) for sample in sample_data})

    # 创建一个新的列表存储结果
    ego_list = []

    for ts in unique_timestamps:
        closest_ego = find_closest_timestamp(float(ts), ego_data)
        # 替换 timestamp，并生成 token
        token = generate_random_token()
        ego_obj = {
            "token": token,
            "timestamp": ts,
            "rotation": closest_ego['rotation'],
            "translation": closest_ego['translation']
        }
        ego_list.append(ego_obj)

    # 打印结果
    # for ego in ego_list:
    #     print(f"Token: {ego['token']}, Timestamp: {ego['timestamp']}, Rotation: {ego['rotation']}, Translation: {ego['translation']}")

    # 输出到 ego_pose.json 文件
    with open('ego_pose.json', 'w') as outfile:
        json.dump(ego_list, outfile, indent=4)

# if __name__ == "__main__":
#     main()

Token: yAvPkXVMzU5fWbaMwdAdNfIrkwhoU5gK, Timestamp: 5408.5, Rotation: [0.15680757835549392, -0.3046518001694224, 0.631587237134952, 0.6954827286942888], Translation: [116.4596746956, 39.7382307567, 22.606]
Token: TxcBfG2FeJ2h1dR7ZPT9twfRfPtuu3cH, Timestamp: 5409.0, Rotation: [0.3396703071756749, -0.15231518906209945, 0.6442581441572982, 0.668098502685909], Translation: [116.4596500183, 39.7383289044, 22.649]
Token: vP5s2a2uUzufd1axFLBPTeYjuHszYt0z, Timestamp: 5409.5, Rotation: [0.23946534007813622, -0.28640507384963115, 0.5777089502591451, 0.7258655890486383], Translation: [116.4596253178, 39.7384256941, 22.608]
Token: YAH9OVh6Lt4wFHdOdShtGySrvjKPzFjM, Timestamp: 5410.0, Rotation: [-0.05939721275489036, -0.1129079574429766, 0.5894905861926268, 0.7976368929865761], Translation: [116.4596005991, 39.7385218868, 22.586]
Token: wWVBBEOoo2HJhDU92FZz7OvkfrG5BtAQ, Timestamp: 5410.5, Rotation: [0.3304314102596638, -0.3820559375795025, 0.5127778303955721, 0.6941953905989775], Translation: [116.4

### rewrite translation and rotation for `ego_pose.json`

In [12]:
import json
import csv
import math
from geopy.distance import geodesic, great_circle
from pyquaternion import Quaternion

def calculate_relative_displacement(reference_point, relative_point):
    lat_displacement = geodesic((reference_point[0], reference_point[1]), (relative_point[0], reference_point[1])).meters
    if relative_point[0] < reference_point[0]:
        lat_displacement = -lat_displacement

    lon_displacement = geodesic((reference_point[0], reference_point[1]), (reference_point[0], relative_point[1])).meters
    if relative_point[1] < reference_point[1]:
        lon_displacement = -lon_displacement

    return [lon_displacement, lat_displacement]

def find_closest_time(imu_data, timestamp):
    # Convert ego_pose timestamp to comparable format
    timestamp_ms = int(float(timestamp) * 1000000)
    closest_entry = min(imu_data, key=lambda x: abs(int(x['systemTime']) - timestamp_ms))
    return closest_entry

def euler_to_quaternion(roll, pitch, heading):
    # Convert degrees to radians
    roll_rad = math.radians(roll)
    pitch_rad = math.radians(pitch)
    heading_rad = math.radians(heading)

    # Use pyquaternion to calculate quaternion
    q = Quaternion(axis=[1, 0, 0], angle=roll_rad) * \
        Quaternion(axis=[0, 1, 0], angle=pitch_rad) * \
        Quaternion(axis=[0, 0, 1], angle=heading_rad)

    return q

def update_json_data(input_json, imu_data, reference_point, output_filename):
    for entry in input_json:
        timestamp = entry['timestamp']
        closest_imu_entry = find_closest_time(imu_data, timestamp)

        # Calculate displacement using IMU latitude and longitude
        relative_point = [float(closest_imu_entry['latitude']), float(closest_imu_entry['longitude'])]
        displacement = calculate_relative_displacement(reference_point, relative_point)

        # Update the translation with displacement values and set height to 0.0
        entry['translation'] = [displacement[1], displacement[0], 0.0]

        # Calculate quaternion from IMU roll, pitch, and heading
        roll = float(closest_imu_entry['roll'])
        pitch = float(closest_imu_entry['pitch'])
        heading = float(closest_imu_entry['heading'])
        quaternion = euler_to_quaternion(roll, pitch, heading)
        
        # Update the rotation with quaternion values
        entry['rotation'] = [quaternion.x, quaternion.y, quaternion.z, quaternion.w]

    with open(output_filename, 'w') as outfile:
        json.dump(input_json, outfile, indent=4)
    print(f"Updated JSON data written to {output_filename}")

if __name__ == '__main__':
    # Load the JSON data from a file
    input_filename = './dataset/MB_dataset2/ego_pose.json'  # Replace with your input file name
    output_filename = 'ego_pose.json'  # Replace with your desired output file name

    with open(input_filename, 'r') as infile:
        json_data = json.load(infile)

    # Load IMU data from the text file
    imu_filename = './dataset/MB_dataset2/dgps_aligned_info.txt'  # Replace with your IMU file name
    imu_data = []
    with open(imu_filename, 'r') as imu_file:
        reader = csv.DictReader(imu_file)
        for row in reader:
            imu_data.append(row)
    reference_point = [39.7346583, 116.4518842]  # Latitude, Longitude
    p1 = [39.7382307567, 116.4596746956]
    relative_point  = [39.7445116, 116.4601607]

    displacement = calculate_relative_displacement(reference_point, relative_point)
    distance = great_circle(reference_point, relative_point).meters
    print(f"Relative displacement: Longitude: {displacement[0]} meters, Latitude: {displacement[1]} meters, straight displacement: {distance} meters")
    update_json_data(json_data, imu_data, reference_point, output_filename)


Relative displacement: Longitude: 709.4902540769446 meters, Latitude: 1094.0082766999883 meters, straight displacement: 1304.312158597344 meters
Updated JSON data written to ego_pose.json


### rewrite `ego_pose_token` for `sample_data.json`

In [10]:
import json
import os

# 文件路径
SAMPLE_DATA_FILE = './dataset/MB_dataset2/mb_test/sample_data.json'
EGO_POSE_FILE = './dataset/MB_dataset2/mb_test/ego_pose.json'
ERROR_LOG_FILE = 'error.log'

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def save_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def find_ego_pose_token_by_timestamp(ego_poses, timestamp):
    for pose in ego_poses:
        if pose['timestamp'] == timestamp:
            return pose['token']
    return None

def update_sample_data_with_ego_pose(sample_data, ego_poses):
    errors = []
    for entry in sample_data:
        timestamp = entry['timestamp']
        new_ego_pose_token = find_ego_pose_token_by_timestamp(ego_poses, timestamp)
        if new_ego_pose_token:
            entry['ego_pose_token'] = new_ego_pose_token
        else:
            error_message = f"Error: No matching ego_pose found for timestamp {timestamp} in sample_data token {entry['token']}"
            print(error_message)
            errors.append(error_message)
    return errors

def write_errors_to_log(errors, log_file):
    with open(log_file, 'w') as f:
        for error in errors:
            f.write(error + "\n")

def main():
    # 加载 JSON 数据
    sample_data = load_json(SAMPLE_DATA_FILE)
    ego_poses = load_json(EGO_POSE_FILE)

    # 更新 sample_data 中的 ego_pose_token
    errors = update_sample_data_with_ego_pose(sample_data, ego_poses)

    # 保存更新后的 sample_data
    save_json(sample_data, SAMPLE_DATA_FILE)

    # 写入错误日志
    if errors:
        write_errors_to_log(errors, ERROR_LOG_FILE)
        print(f"Errors logged to {ERROR_LOG_FILE}")

if __name__ == '__main__':
    main()
